## Setup

In [1]:
import os, sys
sys.path.append(os.path.abspath('../../src/'))

import torch
import torch.optim as optim
import torch.nn as nn
import itertools
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from model.spectogram_dataset import SpectrogramDataset

import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mldogs (mldogs-wut). Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## CNN

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, dropout_conv_p=0.3, dropout_fc_p=0.5, use_batchnorm=True):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # Smaller kernel size
        self.bn1 = nn.BatchNorm2d(16) if use_batchnorm else None
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32) if use_batchnorm else None
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Added layer
        self.bn3 = nn.BatchNorm2d(64) if use_batchnorm else None

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # GAP instead of flattening
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 1)

        self.dropout_conv = nn.Dropout2d(p=dropout_conv_p)
        self.dropout_fc = nn.Dropout(p=dropout_fc_p)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))) if self.bn1 else F.relu(self.conv1(x))
        x = F.relu(self.bn2(self.conv2(x))) if self.bn2 else F.relu(self.conv2(x))
        x = F.relu(self.bn3(self.conv3(x))) if self.bn3 else F.relu(self.conv3(x))
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = torch.sigmoid(self.fc2(x))  # Binary classification
        return x



In [4]:
from sklearn.metrics import f1_score

def train(model, criterion, optimizer, num_epochs, train_loader, val_loader, model_name = "cnn"):
    model.to(device)
    
    PRINT_STEP = len(train_loader) // 5 - 1
    epochs_without_val_acc_improvement = 0
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        model.train()

        running_loss = 0.0
        correct = 0
        all_labels = []
        all_preds = []

        for  i, data in enumerate(train_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = (outputs > 0.5).float()  # Binary prediction with threshold 0.5
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(preds.detach().cpu().numpy().flatten())

            correct += (preds == labels).float().mean().item()
            
            running_loss += loss.item()

            if i % PRINT_STEP == PRINT_STEP-1:
                accuracy = correct / PRINT_STEP
                loss = running_loss / PRINT_STEP
                step = epoch * len(train_loader) + i
                wandb.log({
                        "train/accuracy": accuracy,
                        "train/loss": loss
                    },
                    step=step
                )
                running_loss = 0.0
                correct = 0

        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, F1 Score: {f1:.4f}")

        # Validation
        model.eval()

        val_loss = 0.0
        val_correct = 0

        with torch.no_grad():
            for j, data in enumerate(val_loader):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.unsqueeze(1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Binary predictions
                preds = (outputs > 0.5).float()
                val_correct += (preds == labels).float().mean().item()

        accuracy = val_correct / len(val_loader)
        loss = val_loss / len(val_loader)
        wandb.log({
                "validation/accuracy": accuracy,
                "validation/loss": loss
            },
            step=(epoch + 1) * len(train_loader)
        )

        if accuracy > best_val_acc:
            best_val_acc = accuracy
            epochs_without_val_acc_improvement = 0
        else:
            epochs_without_val_acc_improvement += 1
        if epochs_without_val_acc_improvement >= 10:
            print("8 epochs without a val accuracy improvement. Stopping the train")
            break
        if epoch > 15 and accuracy < 0.9:
            print("Too low accuracy. Stopping the train")
            break

    print("Training complete.")
    
def test(model, test_loader):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on test images: {100 * correct // total} %')

## Data

In [5]:
from utils.dataset_creator import SpecgramsRandomFilter, SpecgramsSilentFilter

DATASETS_PARENT_PATH = 'datasets'
DATASET_PATH = 'dataset'
DATA_DIR = os.path.join('../../', DATASETS_PARENT_PATH, DATASET_PATH)

In [21]:
from prepare_datasets import create_datasets

create_datasets(DATASETS_PARENT_PATH, DATASET_PATH, [
    SpecgramsSilentFilter(0.9)
])

DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    interval_duration=2s
    multithreading=True
Class 0 recordings count: 270
Class 1 recordings count: 630
Image properties:
    width=87px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/1
Finished [900/900]Dataset has been exported. Elapsed time: 63.72191381454468s.
DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    interval_duration=2s
    multithreading=True
Class 0 recordings count: 90
Class 1 recordings count: 210
Image properties:
    width=87px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/1
Finished [300/300]Dataset has been exported. Elapsed

In [7]:
INPUT_WIDTH = 67
INPUT_HEIGHT = 66

LEARNING_RATE = 0.001
LR_DECAY = 0.95
BATCH_SIZE = 64
EPOCHS = 30

LEARNING_RATE_ARRAY = [0.03, 0.0001]
BATCH_SIZE_ARRAY = [32, 128]
DROPOUT_ARRAY = [0, 0.2, 0.4]


## Training

In [ ]:
for lr, b, d1, d2 in itertools.product(LEARNING_RATE_ARRAY, BATCH_SIZE_ARRAY, DROPOUT_ARRAY, DROPOUT_ARRAY):
    print(f"lr:{lr}-b:{b}-d_c:{d1}, d_f:{d2}")
    wandb.init(
        project="iml_m3_wnn",
        config={
            "learning_rate": lr,
            "learning_rate_decay": LR_DECAY,
            "batch_size": b,
            "input_resolution": (INPUT_WIDTH, INPUT_HEIGHT),
            "architecture": "CNN",
            "dataset": "DAPS"
        },
        name=f"lr:{lr}-b:{b}-d_c:{d1}, d_f:{d2}-d"
    )
    model = Net()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    transform = transforms.ToTensor()
    
    train_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "train"), transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    val_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "validation"), transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    test_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "test"),transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    train(model, criterion, optimizer, EPOCHS, train_loader, val_loader, model_name="simple_cnn")

    wandb.finish()


lr:0.03-b:32-d_c:0, d_f:0


Epoch 1/30, Loss: 0.0022, F1 Score: 0.4132
Epoch 2/30, Loss: 0.0022, F1 Score: 0.4106
Epoch 3/30, Loss: 0.0020, F1 Score: 0.4106
Epoch 4/30, Loss: 0.0019, F1 Score: 0.5466
Epoch 5/30, Loss: 0.0020, F1 Score: 0.6744
Epoch 6/30, Loss: 0.0018, F1 Score: 0.7118
Epoch 7/30, Loss: 0.0018, F1 Score: 0.7226
Epoch 8/30, Loss: 0.0018, F1 Score: 0.7325
Epoch 9/30, Loss: 0.0016, F1 Score: 0.7365
Epoch 10/30, Loss: 0.0018, F1 Score: 0.7446
Epoch 11/30, Loss: 0.0019, F1 Score: 0.7492
Epoch 12/30, Loss: 0.0017, F1 Score: 0.7555
Epoch 13/30, Loss: 0.0016, F1 Score: 0.7589
Epoch 14/30, Loss: 0.0015, F1 Score: 0.7630
Epoch 15/30, Loss: 0.0016, F1 Score: 0.7679
Epoch 16/30, Loss: 0.0017, F1 Score: 0.7690
Epoch 17/30, Loss: 0.0018, F1 Score: 0.7697
Too low accuracy. Stopping the train
Training complete.


train/accuracy,▁▁▁▁▁▁▁▁▁▃▄▅▅▅▅▆▆▆▆▇▆▆▆▇▇▆▇▇▇▇▇▇██████▇█
train/loss,█████▇▇▇▇▆▅▄▄▄▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁
validation/accuracy,▃▃▃▅▁▆▅▆▇▆▇▇██▇█▇
validation/loss,█▇▇▅▆▆▄▄▂▇▂▆▂▂▃▂▁
train/accuracy,0.80597
train/loss,0.42702
validation/accuracy,0.78214
validation/loss,0.4485


lr:0.03-b:32-d_c:0, d_f:0.2


Epoch 1/30, Loss: 0.0020, F1 Score: 0.4597
Epoch 2/30, Loss: 0.0019, F1 Score: 0.5679
Epoch 3/30, Loss: 0.0017, F1 Score: 0.6105
Epoch 4/30, Loss: 0.0018, F1 Score: 0.6290
Epoch 5/30, Loss: 0.0018, F1 Score: 0.6496
Epoch 6/30, Loss: 0.0015, F1 Score: 0.6780
Epoch 7/30, Loss: 0.0016, F1 Score: 0.6841
Epoch 8/30, Loss: 0.0015, F1 Score: 0.7229
Epoch 9/30, Loss: 0.0016, F1 Score: 0.7464
Epoch 10/30, Loss: 0.0016, F1 Score: 0.7554
Epoch 11/30, Loss: 0.0017, F1 Score: 0.7508
Epoch 12/30, Loss: 0.0016, F1 Score: 0.7686
Epoch 13/30, Loss: 0.0016, F1 Score: 0.7695
Epoch 14/30, Loss: 0.0016, F1 Score: 0.7789
Epoch 15/30, Loss: 0.0016, F1 Score: 0.7838
Epoch 16/30, Loss: 0.0013, F1 Score: 0.7824
Epoch 17/30, Loss: 0.0017, F1 Score: 0.7898
Too low accuracy. Stopping the train
Training complete.


...

lr:0.03-b:128-d_c:0.2, d_f:0.4


Epoch 1/30, Loss: 0.0021, F1 Score: 0.4695
Epoch 2/30, Loss: 0.0019, F1 Score: 0.5606
Epoch 3/30, Loss: 0.0020, F1 Score: 0.5902
Epoch 4/30, Loss: 0.0020, F1 Score: 0.6009
Epoch 5/30, Loss: 0.0020, F1 Score: 0.6077
Epoch 6/30, Loss: 0.0018, F1 Score: 0.6088
Epoch 7/30, Loss: 0.0019, F1 Score: 0.6143
Epoch 8/30, Loss: 0.0018, F1 Score: 0.6149
Epoch 9/30, Loss: 0.0018, F1 Score: 0.6212
Epoch 10/30, Loss: 0.0019, F1 Score: 0.6185
Epoch 11/30, Loss: 0.0017, F1 Score: 0.6255
Epoch 12/30, Loss: 0.0018, F1 Score: 0.6283
Epoch 13/30, Loss: 0.0019, F1 Score: 0.6303
Epoch 14/30, Loss: 0.0018, F1 Score: 0.6369
Epoch 15/30, Loss: 0.0019, F1 Score: 0.6380
Epoch 16/30, Loss: 0.0018, F1 Score: 0.6490
Epoch 17/30, Loss: 0.0020, F1 Score: 0.6769
Too low accuracy. Stopping the train
Training complete.


train/accuracy,▁▁▂▃▄▄▅▄▄▅▅▅▅▅▅▄▅▅▅▅▅▅▅▅▆▆▆▆▆▅▆▆▆▆▆▇▇▇█▇
train/loss,██▇▆▅▅▅▅▅▄▄▄▄▄▄▃▄▃▃▃▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
validation/accuracy,▂▁▇▇▅▆▁▇█▇██▇▅▁█▇
validation/loss,▁▁▂█▁▁▁▂▁▁▁▁▁▁▁▁▂
train/accuracy,0.76087
train/loss,0.46737
validation/accuracy,0.69805
validation/loss,1.15087


lr:0.03-b:128-d_c:0.4, d_f:0


Epoch 1/30, Loss: 0.0022, F1 Score: 0.4341
Epoch 2/30, Loss: 0.0021, F1 Score: 0.4808


In [17]:
wandb.finish()

train/accuracy,▁
train/loss,▁
train/accuracy,0.70072
train/loss,0.58681
